## Scraping the UC Davis Course Catalog

The 2022-2023 UC Davis course catalog is online at https://catalog.ucdavis.edu/courses-subject-code/.

In this notebook, I will proceed as follows:
   
       1. Use the lxml package to parse these web pages and then retrieve the names of all subject departments, 
       course titles, and course descriptions. 
       2. Use the extracted information to create a data frame and then clean the data. 
   
I will then visualize this data in Tableau. This will provide a summary of what type of courses are offered at UC Davis and what students would be learning in each specific department.

In [2]:
# import necessary packages

import lxml.html as lx
import re
import requests
import requests_cache
import nltk
import pandas as pd

In [66]:
requests_cache.install_cache('cache')

# write function to retrieve all of the subject names in the catalog (for example: 'Chemistry', 'English', etc.)
def retrieve_subjects(url):
    """
    Input: The URL for the UC Davis course catalog home page.
    Output: A list of all academic subjects in the course catalog.
    """
    # download the web page
    r = requests.get(url)
    doc = r.text

    # parse the web page
    html = lx.fromstring(doc, base_url = url)
    subjects = html.xpath('//ul[@id = "/courses-subject-code/"]/li/a')
    subjects = [x.text_content() for x in subjects]
    
    return subjects

In [67]:
# call function to get course departments/subjects
subjects = retrieve_subjects('https://catalog.ucdavis.edu/courses-subject-code/')

# fix strings in subject list
subjects = [x.replace('&\u200b', '&') for x in subjects]

# get subject codes from subject strings
codes = []
for i in subjects:
    # retrieve the substrings inside the parentheses
    code = i[i.find('(')+1:i.find(')')]
    codes.append(code)

# fix string for 'PBI' subject code    
codes = [x.replace('Graduate Group', 'PBI') for x in codes]

# convert all strings to lower case in order to use them later in URLs
codes = [x.lower() for x in codes]

In [229]:
len(subjects)

216

There are 216 different subjects that are offered at the school.

In [184]:
requests_cache.install_cache('cache')

# write function to retrieve all of the course names and descriptions for each subject
def get_course_info(codes, url):
    """
    Input: A list of subject codes and the url for the UC Davis course catalog.
    Output: The course codes, names, and descriptions.
    """
    titles = []
    descriptions = []
    course_codes = []
    
    # iterate through list of subject codes in order to use the URL for each subject
    for i in codes:
        
        # get URL for current subject
        url_sub = url + i + '/'
        
        # download the web page
        r = requests.get(url_sub)
        doc = r.text
        html = lx.fromstring(doc, base_url = url_sub)
        
        # extract course codes for current subject
        crs_codes = html.xpath('//span[@class = "text courseblockdetail detail-code margin--span text--semibold text--big"]')
        crs_codes = [x.text_content() for x in crs_codes]
        course_codes = course_codes + crs_codes
        
        # extract course titles for current subject
        codetitles = html.xpath('//span[@class = "text courseblockdetail detail-title margin--span text--semibold text--big"]')
        codetitles = [x.text_content() for x in codetitles]
        titles = titles + codetitles
        
        # extract course descriptions for current subject
        code_desc = html.xpath('//p[@class = "courseblockextra noindent"]')
        code_desc = [x.text_content() for x in code_desc]
        descriptions = descriptions + code_desc
        
        
    return course_codes, titles, descriptions

In [185]:
url = 'https://catalog.ucdavis.edu/courses-subject-code/'

# call function to get course codes, titles and descriptions for all courses in catalog
course_codes, titles, descriptions = get_course_info(codes, url)

In [230]:
len(course_codes)

9886

There are currently 9886 different courses offered at the school.

In [191]:
# fix strings
titles = [x.replace('—\xa0', '') for x in titles]
descriptions = [x.replace('Course Description:', '') for x in descriptions]

# create data frame
course_df = pd.DataFrame({'course_code': course_codes, 'course': titles, 'descriptions':descriptions})

# view the first 5 rows of the data frame
course_df.head()

,course_code,course,descriptions
0,EAE 001,Introduction to Aerospace Science Engineering,Description of the field of aerospace enginee...
1,EAE 010,From the Wright Brothers to Drones & Quadcopters,History of aircraft and its influence on soci...
2,EAE 099,Special Study for Undergraduates,Special study for undergraduates.
3,EAE 126,Theoretical & Computational Aerodynamics,Development of general equations of fluid mot...
4,EAE 127,Applied Aircraft Aerodynamics,"Principles, governing equations, and predicti..."


In [199]:
subject_codes = []

# use list of all subjects to get the corresponding subject for each course title
for i in course_codes:
    sub_code = i[0:3]
    for j in subjects:
        if sub_code in j:
            subject_code = j
            subject_codes.append(subject_code)

# add course subjects to the data frame
course_df['subject'] = subject_codes

### Using the Course Catalog Data Frame to Create a Data Frame for my Courses

I have a file with the course codes and names for the courses that I took. I will use the course catalog dataframe to retrieve the course descriptions for the courses that I took.

In [219]:
# open file
my_courses = pd.read_excel('courses.xlsx')
my_courses = my_courses.rename(columns = {'Code' : 'course_code', 'Title' : 'course'})

# use course codes to create data frame with course descriptions from course catalog for my courses
my_courses_df = pd.merge(course_df, my_courses, on = ['course_code'])
my_courses_df

,course_code,course_x,descriptions,subject,course_y
0,BIS 002A,Introduction to Biology: Essentials of Life on...,Essentials of life including sources and use ...,Biological Sciences (BIS),Introduction to Biology
1,BIS 010,Everyday Biology,Everyday biological concepts using contempora...,Biological Sciences (BIS),Everyday Biology
2,CHE 002A,General Chemistry,"Periodic table, stoichiometry, chemical equat...",Chemistry (CHE),General Chemistry
3,CHE 002B,General Chemistry,"Continuation of CHE 002A. Thermodynamics, ato...",Chemistry (CHE),General Chemistry
4,CMN 003,Interpersonal Communication Competence,Communication competence in professional sett...,Communication (CMN),Interpersonal Communication Competence
5,ECS 020,Discrete Mathematics For Computer Science,Discrete mathematics of particular utility to...,Computer Science Engineering (ECS),Discrete Mathematics for Computer Science
6,ECS 032A,Introduction to Programming,Introduction to programming and problem solvi...,Computer Science Engineering (ECS),Introduction to Programming
7,ECS 034,Software Development in UNIX & C++,UNIX Operating system tools and programming e...,Computer Science Engineering (ECS),Programming & Problem Solving
8,ECS 036B,Software Development & Object-Oriented Program...,Object-oriented programming in C++. Basic dat...,Computer Science Engineering (ECS),Software Development & Object-Oriented Program...
9,ECS 036C,"Data Structures, Algorithms, & Programming",Design and analysis of data structures for a ...,Computer Science Engineering (ECS),"Data Structures, Algorithms, and Programming"


## Removing Noise From the Text Descriptions

Common words such as "to" and "of" are not useful for understanding the content of the course descriptions. Before creating word clouds from the course descriptions, I want to remove these words from the 'descriptions' column. 

In [223]:
# remove noise from the text descriptions
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords = stopwords.words('english')

# remove stopwords from all course descriptions
course_df['descriptions'] = course_df['descriptions'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
    
my_courses_df['descriptions'] = my_courses_df['descriptions'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Natalie\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [243]:
# remove more characters from course descriptions
course_df['descriptions'] = course_df['descriptions'].str.replace('&','')
my_courses_df['descriptions'] = my_courses_df['descriptions'].str.replace('&','')
course_df['descriptions'] = course_df['descriptions'].str.replace(',','')
my_courses_df['descriptions'] = my_courses_df['descriptions'].str.replace(',','')
course_df['descriptions'] = course_df['descriptions'].str.replace(':','')
my_courses_df['descriptions'] = my_courses_df['descriptions'].str.replace(':','')
course_df['descriptions'] = course_df['descriptions'].str.replace('.','')
my_courses_df['descriptions'] = my_courses_df['descriptions'].str.replace('.','')
course_df['descriptions'] = course_df['descriptions'].str.replace(';','')
my_courses_df['descriptions'] = my_courses_df['descriptions'].str.replace(';','')
course_df['descriptions'] = course_df['descriptions'].str.replace('/','')
my_courses_df['descriptions'] = my_courses_df['descriptions'].str.replace('/','')

# convert all course description text to lower case
course_df['descriptions'] = course_df['descriptions'].str.lower()
my_courses_df['descriptions'] = my_courses_df['descriptions'].str.lower()

In [244]:
# write data frames to csv files

course_df.to_csv('course_catalog.csv')
my_courses_df.to_csv('my_courses.csv')

The Tableau visualization of the course description data can be found at: https://public.tableau.com/app/profile/natalie.perillo/viz/UCDavisCourseCatalog/Dashboard1